## Module 2: Convolutional Neural Networks (CNNs/ConvNets)

## Convolutional Neural Networks: Architectures, Convolution / Pooling Layers
Similar to ordinary Neural Networks. They are made up of neurons that have learnable weights and biases. Each neuron receives some inputs, performs a dot product and optionally follows it with a non-linearity. The whole network still expresses a single differentiable score function: from the raw image pixels on one end to class scores at the other. And they still have a loss function (SVM/Softmax) on the last (fully-connected) layer. ConvNet architectures make the explicit assumption that the inputs are images.<br>

### Architecture Overview
Recall: The regular neural network take the input (single vector), transform by using lots of hidden layers, made up by independent and fully connected neurons. The last fully-connected layer is claaed the "output layer". But this full connectivity is wasteful and huge number of parameter will quickly lead to overfitting.<br>

For the CNN, the neurons in a layer will only be connected to a small region of the layer before it. At the end of the ConvNet arch. we will reduce the full image into a single vector of class scores. Can visu.:<br>
![cscnn1](/notebooks/imgs/cscnn1.jpg)<br>
<i><strong>A ConvNet is made up of Layers. Every Layer has a simple API: it transforms an input 3D volume to an output 3D volume w/ some differentiable function that may or may not have parameters. </i></strong><br>

### Layers used to build ConvNets
3 Main types of layers: <strong>Convolutiona Layer, Pooling Layer, Fully-Connected Layer</strong>(same as the regular neural networks)<br>

Simple ConvNet for CIFAR-10 classification with the arch. [Input - CONV- RELU - POOL - FC]:<br>
1. Input: $32*32*3$, raw pixel values of the image.<br>
2. CONV layer compute the output of neurons connected to local regions in the input. Dot product between weights and small region they connected to.<br>
3. RELU layer apply elementwise activation function, such as the $\max(0,x)$ thresholding at zero. Same size as before.<br>
4. POOL layer perform downsampling operation along the spatial dimension<br>
5. FC (fully-connected) layercompute the class scores. Each neuron in this layer will be connected to all the numbers in the previous volume.<br>

The CONV/FC layers perform transformation that are a function of not only the activations in the input volume, but also of the parameters. The RELU/POOL layers implement a fixed function. Tha parameters in the CONV/FC trained w/ GD so the class scores the ConvNet conputes are consistent w/ the labels in the training set foreach image.<br>

Summary:<br>
1. ConvNet arch. is in the simplest case a list of Layers that transform the image volume into an output volume.<br>
2. There are a few distinct types of Layers ()CONV/FC/RELU/POOL etc.)<br>
3. Each Layer accepts an input #D volume and transforms it to an output 3D volume through a differentiable function<br>
4. Each Layer may or may not have parameters<br>
5. Each Layer may or may not have addtional hyperparameters<br>

![cscnn2](/notebooks/imgs/cscnn2.jpg)<br>

### Convolutional Layer
<strong>Overview and intuition without brain stuff</strong>.<br>
The CONV layer's parameters consist of a set of learnable filters. Erery filter is small spatially, but extends through the full depth of the input volume. During the forward pass, slide (convolve) each filter across the width and height of the input volume and compute dot products between the entries of the filter and the input at any position. Durning the process, we produce 2-D activation map gives the responses of that filter at every spatial position. The network learn filters that activate when they see some type of visual feature such as an edge of some orientation or a blotch of some color on the first layer, or eventually entire honeycomb or wheel-like patterns on higher layers of the nerwork.<br>
<strong>Local Connectivity</strong>.<br>
When high-dimension, we connect each neuron to only a local region of the input volume. The spatial extent of this connectivity is a hyperparameter called the <strong>receptive field</strong> of the neuron.
![cscnn3](/notebooks/imgs/cscnn3.jpg)<br>


<strong>Spatial arrangement</strong><br>
3 hpyerpatameteres control the size of the ouput volume: <strong>depth, stride, zero-padding</strong>.<br>
1. <strong>Depth</strong>. Corresponds to the # of filters we like to use, each learning to look different thing in the input. Refer to a set of neurons that are all looking at the same region of the input as a <strong> depth column</strong><br>
2. <strong>Stride</strong>. When stride is 1 then move the filters one pixel at a time. When 2, jump 2 pixels at a time when slide around, which help produce smaller output volumes spatially, unusual to use 3 or more.<br>
3. <strong>zero-padding</strong>. Sometimes, it will be convenient to pad the input volume w. zeros around the border. The zero padding allow us to control the spatial size of the output volumes.<br>

We can compute the spatial size of the output volume as a function fo the input volume size ($W)$, the receptive field size ($F$). The stride ($S$) and the amount of zero padding ($P$) on the border. Calculating how many neurons 'fit' is:<br>
$$(W-F+2P)/S+1$$<br>
![cscnn4](/notebooks/imgs/cscnn4.jpg)<br>
When the stride $S=1$ and setting zero padding to be $P=(F-1)/2$ ensures the input volume and output volume have the same size spatially.<br>

<strong>Prameter Sharing</strong>.<br>
Use to control the number of parameters. We made the following assumption: if one feature is useful to compute at some spatial position, then it should also be useful to compute at a different position. Denoting a single 2-dimensional slice of depth as a <strong>depth slice</strong>, we are going to constrain the neurons in each depth slice to use the same weights and bias.<br>
Notice if all neurons in a single depth slice are using the same weight vector, then the forward pass of the CONV layer cna in each depth slice be computed as a convolution of the neuron's weights with the input volum.
![cscnn5](/notebooks/imgs/cscnn5.jpg)<br>